In [1]:
# Importar las librerías necesarias
import pandas as pd
import numpy as np
import pickle

In [2]:
# Leer el dataset
df = pd.read_csv("datasets/MasterBI_Test.csv", header=0, sep=",", encoding="utf-8")
# Crear un dataframe nuevo con en MachineIdentifier y luego adicionarle las probabilidades predichas por el modelo
kaggle_submission = df[['MachineIdentifier']]

# Mostrar todas las columnas del dataframe
pd.set_option('display.max_columns',None)
df.head()


C:\Users\martin.salas\AppData\Local\Temp\ipykernel_5412\1469484182.py:2: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("datasets/MasterBI_Test.csv", header=0, sep=",", encoding="utf-8")


,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,AVProductsInstalled,AVProductsEnabled,HasTpm,CountryIdentifier,CityIdentifier,OrganizationIdentifier,GeoNameIdentifier,LocaleEnglishNameIdentifier,Platform,Processor,OsVer,OsBuild,OsSuite,OsPlatformSubRelease,OsBuildLab,SkuEdition,IsProtected,AutoSampleOptIn,PuaMode,SMode,IeVerIdentifier,SmartScreen,Firewall,UacLuaenable,Census_MDC2FormFactor,Census_DeviceFamily,Census_OEMNameIdentifier,Census_OEMModelIdentifier,Census_ProcessorCoreCount,Census_ProcessorManufacturerIdentifier,Census_ProcessorModelIdentifier,Census_ProcessorClass,Census_PrimaryDiskTotalCapacity,Census_PrimaryDiskTypeName,Census_SystemVolumeTotalCapacity,Census_HasOpticalDiskDrive,Census_TotalPhysicalRAM,Census_ChassisTypeName,Census_InternalPrimaryDiagonalDisplaySizeInInches,Census_InternalPrimaryDisplayResolutionHorizontal,Census_InternalPrimaryDisplayResolutionVertical,Census_PowerPlatformRoleName,Census_InternalBatteryType,Census_InternalBatteryNumberOfCharges,Census_OSVersion,Census_OSArchitecture,Census_OSBranch,Census_OSBuildNumber,Census_OSBuildRevision,Census_OSEdition,Census_OSSkuName,Census_OSInstallTypeName,Census_OSInstallLanguageIdentifier,Census_OSUILocaleIdentifier,Census_OSWUAutoUpdateOptionsName,Census_IsPortableOperatingSystem,Census_GenuineStateName,Census_ActivationChannel,Census_IsFlightingInternal,Census_IsFlightsDisabled,Census_FlightRing,Census_ThresholdOptIn,Census_FirmwareManufacturerIdentifier,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier
0,961e21282c5c1474ce2330d8d528be6d,win8defender,1.1.15200.1,4.10.209.0,1.275.1058.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,171,128117.0,18.0,211.0,182,windows8,x64,6.3.0.0,9600,768,windows8.1,9600.19101.amd64fre.winblue_ltsb_escrow.180718...,Home,1.0,0,NaN,0.0,333.0,RequireAdmin,1.0,1.0,Notebook,Windows.Desktop,4142.0,152126.0,4.0,1.0,256.0,NaN,715404.0,HDD,691538.0,0,8192.0,Laptop,14.0,1366.0,768.0,Mobile,lion,0.000000e+00,10.0.10586.318,amd64,th2_release,10586,318,CoreSingleLanguage,CORE_SINGLELANGUAGE,Upgrade,29.0,125,FullAuto,0,IS_GENUINE,Retail,NaN,NaN,Retail,0.0,142.0,52290.0,1,0.0,0.0,0,0,0.0,0.0,3.0
1,aa943fbc7cb6b6a77bdf7f61a1ba310c,win8defender,1.1.15200.1,4.18.1807.18075,1.275.727.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,119,85981.0,14.0,98.0,103,windows10,x64,10.0.0.0,17134,768,rs4,17134.1.amd64fre.rs4_release.180410-1804,Home,1.0,0,NaN,0.0,137.0,NaN,1.0,1.0,Desktop,Windows.Desktop,5771.0,328142.0,4.0,5.0,2948.0,NaN,32768.0,UNKNOWN,32217.0,0,2047.0,Other,22.4,1920.0,975.0,Desktop,NaN,4.294967e+09,10.0.17134.228,amd64,rs4_release,17134,228,Core,CORE,IBSClean,5.0,26,FullAuto,0,INVALID_LICENSE,Retail,NaN,0.0,Retail,NaN,NaN,NaN,0,NaN,1.0,0,0,0.0,0.0,15.0
2,917cad1527312b93d21d95cf35355be7,win8defender,1.1.14800.3,4.9.10586.672,1.267.1727.0,0,7.0,0,NaN,61343.0,2.0,1.0,1,159,66064.0,27.0,276.0,74,windows10,x64,10.0.0.0,10586,768,th2,10586.672.amd64fre.th2_release_sec.161024-1825,Home,0.0,0,NaN,0.0,87.0,RequireAdmin,1.0,1.0,Notebook,Windows.Desktop,2102.0,242491.0,4.0,5.0,3397.0,NaN,1907729.0,HDD,1890123.0,0,8192.0,Notebook,15.5,1366.0,768.0,Mobile,lion,0.000000e+00,10.0.10586.753,amd64,th2_release_sec,10586,753,Core,CORE,Other,7.0,30,UNKNOWN,0,IS_GENUINE,OEM:DM,0.0,0.0,Retail,0.0,554.0,33084.0,1,0.0,0.0,0,0,0.0,0.0,11.0
3,394f0ad0dbbc63d71940f11086388619,win8defender,1.1.15100.1,4.10.209.0,1.273.703.0,0,7.0,0,NaN,53447.0,1.0,1.0,1,164,82904.0,18.0,205.0,172,windows8,x64,6.3.0.0,9600,768,windows8.1,9600.19067.amd64fre.winblue_ltsb_escrow.180619...,Home,1.0,0,NaN,0.0,333.0,RequireAdmin,1.0,1.0,Notebook,Windows.Desktop,4490.0,307797.0,4.0,5.0,2535.0,NaN,715404.0,HDD,686001.0,0,6144.0,Notebook,14.0,1366.0,768.0,Mobile,lion,0.000000e+00,10.0.10586.1106,amd64,th2_release_sec,10586,110

In [3]:
# Definir las variables identifier y target
identifier = 'MachineIdentifier' #no aporta valor al modelo
target = 'HasDetections' #variable objetivo

# Eliminar la columna 'MachineIdentifier' del dataframe
df.drop('MachineIdentifier', axis=1, inplace=True)

In [4]:
# Llenar los Nulls de la variable 'Census_IsWIMBootEnabled' con 1 
df['Census_IsWIMBootEnabled'] = df['Census_IsWIMBootEnabled'].fillna(1)

# Llenar los Nulls de la variable 'PuaMode' con 1 
df['PuaMode'] = df['PuaMode'].fillna('off')
df['PuaMode'] = df['PuaMode'].replace({'off': 0, 'on': 1})

C:\Users\martin.salas\AppData\Local\Temp\ipykernel_5412\2029929336.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['PuaMode'] = df['PuaMode'].replace({'off': 0, 'on': 1})


In [5]:
# Cargar la clasificación de las variables (cat, bin, num) desde el archivo pickle
with open("files/clasificacion_vars.pkl", "rb") as f:
    clasificacion_vars = pickle.load(f)

# Unir todas las columnas válidas
columnas_validas = set(clasificacion_vars['binary_cols'] + clasificacion_vars['cat_cols'] + clasificacion_vars['num_cols'])

# Filtrar el DataFrame para conservar solo las columnas válidas
df = df[[col for col in df.columns if col in columnas_validas]]


In [6]:
# Función para aplicar imputaciones guardadas a un nuevo DataFrame
def aplicar_imputaciones(df, archivo_pkl):
    # Cargar el diccionario de imputación
    with open(archivo_pkl, "rb") as f:
        valores_imputacion = pickle.load(f)

    # Aplicar imputaciones binarias
    for col, valor in valores_imputacion.get('binarias', {}).items():
        if isinstance(valor, dict):
            # Imputación probabilística (no reproducible exactamente)
            probs = valor
            df[col] = df[col].apply(lambda x: np.random.choice([0, 1], p=[probs.get(0, 0.5), probs.get(1, 0.5)]) if pd.isnull(x) else x)
        else:
            df[col] = df[col].fillna(valor)

    # Aplicar imputaciones categóricas
    for col, valor in valores_imputacion.get('categoricas', {}).items():
        df[col] = df[col].fillna(valor)

    # Aplicar imputaciones numéricas
    for col, valor in valores_imputacion.get('numericas', {}).items():
        df[col] = df[col].fillna(valor)
        
    return df

df = aplicar_imputaciones(df, 'files/valores_imputacion.pkl')

In [7]:
X = df.copy()

# Cargar el Target Encoder usado en los datos de entrenamiento desde el archivo pickle
with open("files/target_encoder.pkl", "rb") as archivo:
    te = pickle.load(archivo)


for col in te.feature_names_in_:
    if col not in X.columns:
        X[col] = "missing" 

# Comprobar que las columnas categóricas del Target Encoder coinciden con las columnas categóricas del DataFrame
# Si hay alguna diferencia, imprimirla
te_set = set(te.feature_names_in_)
cat_set = set(clasificacion_vars['cat_cols'])
print(te_set - cat_set)
print(cat_set - te_set)

# Añadir las columnas codificadas por Target Encoding al DataFrame original
X_te= X.join(te.transform(X[clasificacion_vars['cat_cols']]).add_suffix('_target'))
# Eliminar las columnas categóricas originales
X_te = X_te.drop(clasificacion_vars['cat_cols'], axis=1)


set()
set()


In [8]:
X = X_te.copy()

# Cargar las variables finales con las que se debe ejecutar el modelo desde el archivo pickle
with open("files/final_features.pkl", "rb") as f:
    final_features = pickle.load(f)

In [9]:
# Filtrar el DataFrame para conservar solo las columnas finales
X = X.loc[:, final_features]

# Guardar datos de test en un diccionario
datos_test = {
    'X': X,
    'kaggle_submission': kaggle_submission
}

# Guardar el diccionario con los datos de test en un archivo pickle
with open("files/datos_test.pkl", "wb") as f:
    pickle.dump(datos_test, f)

X.head()


,SmartScreen_target,AVProductStatesIdentifier_target,Census_IsVirtualDevice,Census_InternalBatteryNumberOfCharges_target,Census_ProcessorModelIdentifier_target,Wdft_IsGamer,AvSigVersion_target,AVProductsInstalled_target,AppVersion_target,Census_HasOpticalDiskDrive,RtpStateBitfield_target,Census_PrimaryDiskTotalCapacity_target,Census_InternalPrimaryDiagonalDisplaySizeInInches_target,EngineVersion_target,Processor_target,Census_OSInstallTypeName_target,Census_OSArchitecture_target,Census_OSVersion_target,CountryIdentifier_target,Census_OEMNameIdentifier_target,SMode
0,0.455631,0.556043,0.0,0.498497,0.668067,0.0,0.480749,0.548222,0.504122,0,0.502941,0.504564,0.504734,0.490845,0.511677,0.473995,0.511565,0.511728,0.494133,0.467699,0.0
1,0.455631,0.556043,1.0,0.517103,0.549162,0.0,0.494171,0.548222,0.529990,0,0.502941,0.117978,0.210530,0.490845,0.511677,0.523276,0.511565,0.531869,0.334667,0.162037,0.0
2,0.455631,0.328932,0.0,0.498497,0.500720,0.0,0.319901,0.395023,0.446396,0,0.502941,0.545361,0.511149,0.311826,0.511677,0.479404,0.511565,0.441698,0.552289,0.517345,0.0
3,0.455631,0.556043,0.0,0.498497,0.515340,0.0,0.542923,0.548222,0.504122,0,0.502941,0.504564,0.504734,0.553219,0.511677,0.478128,0.511565,0.484456,0.483419,0.471662,0.0
4,0.808398,0.506456,0.0,0.498497,0.547674,0.0,0.598381,0.395023,0.529990,0,0.378057,0.529613,0.455043,0.553219,0.511677,0.523276,0.511565,0.485686,0.515999,0.511855,0.0
